In [1]:
import pandas as pd
import numpy as np
import math
from collections import defaultdict
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns', 100)

In [2]:
reg_form  = pd.read_csv("registration_form_all.csv", dtype={'clspcode': str})
reg_form = reg_form.drop(['Unnamed: 0'], axis = 1)
reg_form_clsp = reg_form[['clspcode']]

In [3]:
# fucntion to remove duplicates in a whole wave/survey. If there are duplicates, it looks at the last_page
# variable and only keeps the highest one. 
def remove_duplicates(wave):
    counter = defaultdict(int)
    for pcode in wave['clspcode'].values:
        counter[pcode] += 1

    duplicates = dict(filter(lambda entry: entry[1] >  1, counter.items()))
    total_pcodes = sum([v for _, v in duplicates.items()])
    drop = []
    for double in duplicates.keys():
        max_last_page = wave.loc[wave['clspcode'] == double]['lastpage'].idxmax()
        doubles = wave.loc[wave['clspcode'] == double].index.to_list()
        doubles.remove(max_last_page)
        drop = drop + doubles
    return wave.drop(wave.index[drop])

### Loading Conveniance and Longitudinal

In [4]:
# When loading a survey, make sure of a few things
# 1: Import the clspcode as a string to avoid rounding and scientific notation
# 2: remove entries that do no have clsp codes
# 3: keep only entries that have clspcodes that exist in the registration form
# 4: Remove duplicate clspcpdes using remove_duplicates function (keeps entries with higher last page)
def load_survey(csv_paths: list):
    df = None
    for csv_path in csv_paths:
        if df is None: df = pd.read_csv(csv_path, dtype={'clspcode': str}) 
        else: df = df.append(pd.read_csv(csv_path,dtype={'clspcode': str}), ignore_index=True)
    df = df.loc[~df['clspcode'].isna()]
    df = pd.merge(df, reg_form_clsp, how = 'inner', on=["clspcode"])
    df = remove_duplicates(df)
#     print(df.shape)
    return df

paths = {"convenience7": 
             ["Longitudinal_data_2022_02_11/convenience 7/results-survey157642.csv",
              "Longitudinal_data_2022_02_11/convenience 7/results-survey378275.csv",
              "Longitudinal_data_2022_02_11/convenience 7/results-survey713768.csv",
              "Longitudinal_data_2022_02_11/convenience 7/results-survey882533.csv"],
         
         "longi8": 
             ["Longitudinal_data_2022_02_11//longi 8//results-survey241117.csv",
              "Longitudinal_data_2022_02_11//longi 8//results-survey781799.csv"],
         
         "convenience8": 
             ["Longitudinal_data_2022_02_11/convenience 8/results-survey252679.csv",
              "Longitudinal_data_2022_02_11/convenience 8/results-survey436966.csv",
              "Longitudinal_data_2022_02_11/convenience 8/results-survey714887.csv",
              "Longitudinal_data_2022_02_11/convenience 8/results-survey732215.csv"],

          "longi9":
             ["Longitudinal_data_2022_02_11//longi 9//results-survey138447.csv",
              "Longitudinal_data_2022_02_11//longi 9//results-survey957625.csv"],
         
         "convenience9_10":
             ["Longitudinal_data_2022_02_11/convenience 9-10/results-survey145697.csv",
              "Longitudinal_data_2022_02_11/convenience 9-10/results-survey344374.csv",
              "Longitudinal_data_2022_02_11/convenience 9-10/results-survey394719.csv",
              "Longitudinal_data_2022_02_11/convenience 9-10/results-survey823221.csv"],
         
         "longi10" :
             ["Longitudinal_data_2022_02_11//longi 10//results-survey664739.csv",
              "Longitudinal_data_2022_02_11//longi 10//results-survey924929.csv"],
         
         "convenience11": 
             ["Longitudinal_data_2022_02_11/convenience 11/results-survey168675.csv",
              "Longitudinal_data_2022_02_11/convenience 11/results-survey188396.csv",
              "Longitudinal_data_2022_02_11/convenience 11/results-survey589624.csv",
              "Longitudinal_data_2022_02_11/convenience 11/results-survey832133.csv"],
         
         "longi11" :
             ["Longitudinal_data_2022_02_11//longi 11//results-survey426662.csv",
              "Longitudinal_data_2022_02_11//longi 11//results-survey434562.csv"],
         
         "convenience12_13": 
             ["Longitudinal_data_2022_02_11/convenience 12-13/results-survey254711.csv",
              "Longitudinal_data_2022_02_11/convenience 12-13/results-survey353538.csv",
              "Longitudinal_data_2022_02_11/convenience 12-13/results-survey767924.csv",
              "Longitudinal_data_2022_02_11/convenience 12-13/results-survey821816.csv"],
         
         "longi12" : 
             ["Longitudinal_data_2022_02_11//longi 12//results-survey843852.csv",
              "Longitudinal_data_2022_02_11//longi 12//results-survey964975.csv"],
         
         "convenience14_15": 
             ["Longitudinal_data_2022_02_11/convenience 14-15/results-survey179777.csv",
              "Longitudinal_data_2022_02_11/convenience 14-15/results-survey531246.csv",
              "Longitudinal_data_2022_02_11/convenience 14-15/results-survey943941.csv",
              "Longitudinal_data_2022_02_11/convenience 14-15/results-survey984237.csv"],
         
         "longi14" : 
             ["Longitudinal_data_2022_02_11//longi 14//results-survey361153.csv",
              "Longitudinal_data_2022_02_11//longi 14//results-survey585971.csv"],
         
         "longi15" : 
             ["Longitudinal_data_2022_02_11//longi 15//results-survey521968.csv",
              "Longitudinal_data_2022_02_11//longi 15//results-survey666428.csv"]
        }

df_dict = {}

for df_name, df_paths_to_load in paths.items():
    df_dict[df_name] = load_survey(df_paths_to_load)


### Appending surveys for long merge

In [5]:
# Merging by appending each wave on top of one another. Same variable naems will be stacked on top of each other
# and new variables will create new columns, with NaN values in other cells
long_merge = pd.DataFrame()
for df in df_dict:
    long_merge = long_merge.append(df_dict[df], ignore_index = True)


In [6]:
# Adding in the info from the clsp form (email, + other variables)
long_merge = pd.merge(long_merge, reg_form, how="inner", on=["clspcode"])

In [7]:
long_merge['clspcode'].nunique() # we have 1764 unqiue clsp codes

1764

In [8]:
long_merge.to_csv('long_merge.csv',encoding='utf-8-sig') #saving file

### Appending surveys for wide merge

#### Adding a wave variable to seperate the data by wave

In [9]:
# some surveys had > 1 waves mixed into them. Based on clsp code, we can seperate these out into their appropriate wave 
def get_wave_1_digit(clsp): return int((str(clsp)[0:1]))
def get_wave_2_digit(clsp): return int((str(clsp)[0:2]))

In [10]:
# calls the function defines above to seperate out the waves
df_dict['convenience9_10']['wave'] = np.vectorize(get_wave_1_digit)(df_dict['convenience9_10']['clspcode'])
df_dict['convenience12_13']['wave'] = np.vectorize(get_wave_2_digit)(df_dict['convenience12_13']['clspcode'])
df_dict['convenience14_15']['wave'] = np.vectorize(get_wave_2_digit)(df_dict['convenience14_15']['clspcode'])

In [12]:
# grouping similar waves together from one survey/dataset 
def split_by_wave(df,wavex,wavey):
    by_wave = df_dict[df].groupby('wave')
    df_dict['convenience' + str(wavex)] = by_wave.get_group(wavex).drop(['wave'], axis = 1)
    df_dict['convenience' + str(wavey)] = by_wave.get_group(wavey).drop(['wave'], axis = 1)
    del df_dict[df]

In [12]:
# splitting the dataframes
split_by_wave('convenience9_10', 9, 1)
split_by_wave('convenience12_13', 12, 13)
split_by_wave('convenience14_15', 14, 15)
df_dict['convenience10'] = df_dict.pop('convenience1') # adjsuting wave name

#### Appending same waves together (ie: conveniance 8 + longi 8 = wave 8)

In [14]:
wave7 = df_dict['convenience7']
wave8 = df_dict['convenience8'].append(df_dict['longi8'], ignore_index=True)
wave9 = df_dict['convenience9'].append(df_dict['longi9'], ignore_index=True)
wave10 = df_dict['convenience10'].append(df_dict['longi10'], ignore_index=True)
wave11 = df_dict['convenience11'].append(df_dict['longi11'], ignore_index=True)
wave12 = df_dict['convenience12'].append(df_dict['longi12'], ignore_index=True)
wave13 = df_dict['convenience13']
wave14 = df_dict['convenience14'].append(df_dict['longi14'], ignore_index=True)
wave15 = df_dict['convenience15'].append(df_dict['longi15'], ignore_index=True)

wave_dict = {'wave_7': wave7, 'wave_8': wave8, 'wave_9': wave9, 'wave_10': wave10, 'wave_11': wave11, 
             'wave_12': wave12, 'wave_13': wave13, 'wave_14': wave14, 'wave_15': wave15}
# adding a prefix to each wave. so each wave will have a _waveNumber at the end of each variable
for wave in wave_dict:
    prefix = wave[-2:]
    if prefix[0] !='_': prefix = "_" +prefix
    wave_dict[wave] = wave_dict[wave].rename(columns={col: col+ prefix for col in wave_dict[wave].columns if col not in ['clspcode']})

#### Merging all waves together on clsp codes (outer merge) 

In [20]:
#wide merge by doing an outer merge
wide_merge = pd.DataFrame()
for wave in wave_dict:
    if wide_merge.empty: 
        wide_merge = wave_dict[wave]
        continue
    wide_merge=pd.merge(wide_merge, wave_dict[wave], how="outer", on=["clspcode"])
wide_merge = wide_merge.drop([736], axis = 0)
wide_merge = pd.merge(wide_merge, reg_form, how="inner", on=["clspcode"])

In [21]:
wide_merge.shape #this should match the number of unique clsp codes from the long merge, since this has one row per person.

(1764, 2511)

In [18]:
wide_merge.to_csv('wide_merge.csv',encoding='utf-8-sig') # saving